In [76]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [77]:
# 파일 로드
df_welfare = pd.read_csv('./geocode_welfare_transformed.csv')
df_safety_sign = pd.read_csv('./서울시 안전표지 관련 정보.csv',encoding='euc-kr')

C:\Users\yeeun\AppData\Local\Temp\ipykernel_10192\745944131.py:2: DtypeWarning: Columns (0,1,10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='euc-kr')


In [ ]:
# 좌표 데이터를 Shapely Point 객체로 변환
gdf_welfare = gpd.GeoDataFrame(df_welfare, geometry=[Point(xy) for xy in zip(df_welfare['X'], df_welfare['Y'])])
gdf_safety_sign = gpd.GeoDataFrame(df_safety_sign, geometry=[Point(xy) for xy in zip(df_safety_sign['X좌표'], df_safety_sign['Y좌표'])])

In [ ]:
# GeoDataFrame에 좌표계 설정
gdf_welfare.set_crs(epsg=5186, inplace=True)
gdf_safety_sign.set_crs(epsg=5186, inplace=True)

비고           X좌표           Y좌표                       geometry
0             197430.82666  551261.04855  POINT (197430.827 551261.049)
1       T형설치  198212.57833  553261.42560  POINT (198212.578 553261.426)
2             200019.71305  554940.85846  POINT (200019.713 554940.858)
3             199996.21210  554853.25479  POINT (199996.212 554853.255)
4             199994.21744  554963.77265  POINT (199994.217 554963.773)
...      ...           ...           ...                            ...
349995        200206.09163  549129.25680  POINT (200206.092 549129.257)
349996        199043.00000  551338.78125  POINT (199043.000 551338.781)
349997        198998.59314  551529.46674  POINT (198998.593 551529.467)
349998        198959.17804  551573.54977  POINT (198959.178 551573.550)
349999        198959.73857  551572.89364  POINT (198959.739 551572.894)

[350000 rows x 4 columns]

In [ ]:
# 복지시설의 각 위치에 대해 300미터 버퍼 생성
gdf_welfare['buffer'] = gdf_welfare.geometry.buffer(300)

In [ ]:
# traffic equipment 개수를 계산하는 함수
def count_facilities_within_buffer(row, gdf_b):
    # 버퍼 내의 traffic equipment 필터링
    facilities_within_buffer = gdf_b[gdf_b.geometry.within(row.buffer)]
    return len(facilities_within_buffer)

In [ ]:
# '안전표지 개수' 열 추가
gdf_welfare['안전표지 개수'] = gdf_welfare.apply(lambda row: count_facilities_within_buffer(row, gdf_safety_sign), axis=1)

In [ ]:
safety_sign_count = gdf_welfare[['시설명', '안전표지 개수']].copy()
print(safety_sign_count)


                  시설명  B시설 개수
0            송파구방이복지관     294
1          서울시각장애인복지관     318
2        서울시립발달장애인복지관      10
3               청음복지관     278
4          마포장애인종합복지관     196
..                ...     ...
672     종로발달장애인평생교육센터     544
673  영등포구 발달장애인평생교육센터     411
674     강남발달장애인평생교육센터     409
675    도봉발달장애인 평생교육센터     255
676  용산구 발달장애인 평생교육센터     221

[677 rows x 2 columns]
